In [1]:
import torch
import segmentation_models_pytorch as smp
from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning.logging import TestTubeLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from omegaconf import OmegaConf 
from src.conf.conf_schema import Schema
from src.conf.config import CONFIG
from src.dl.lightning_model import SegModel

/home/leos/.local/lib/python3.6/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

# Train a pytorch model

- Any model should work if it's wrapped to lightning SegModel wrapper. e.g. smp models

In [2]:
model = smp.Unet(
    encoder_name="resnext50_32x4d", 
    classes=2
)

# model = smp.FPN(
#     encoder_name="resnext50_32x4d", 
#     classes=2, 
#     decoder_merge_policy='cat'
# )

In [3]:
config = OmegaConf.merge(Schema, CONFIG)
lightning_model = SegModel.from_conf(model, config)

# Set up lightning trainers and loggers 

In [4]:
tt_logger = TestTubeLogger(
    save_dir=config.experiment_args.experiment_root_dir,
    name=config.experiment_args.model_name,
    version=config.experiment_args.experiment_version
)

checkpoint_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
)

checkpoint_callback = ModelCheckpoint(
    filepath = str(checkpoint_dir),
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'avg_val_loss',
    mode = 'min',
    prefix = ''
)

if config.training_args.resume_training:   
    last_checkpoint_path = lightning_model.fm.model_checkpoint("last")
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        resume_from_checkpoint=str(last_checkpoint_path),
        profiler=True,
    )
    
else:
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        profiler=True,
    )
    

# find the batch size automatically
# new_batch_size = trainer.scale_batch_size(lightning_model)

# Override old batch size
# lightning_model.batch_size = new_batch_size

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning:

Checkpoint directory /home/local/leos/Dippa_test/result/Unet/version_test_pannuke_ddd exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [5]:
log_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
    / "tf"
)

In [6]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Reusing TensorBoard on port 6006 (pid 19547), started 0:06:09 ago. (Use '!kill 19547' to kill it.)

# Train the model

In [ ]:
trainer.fit(lightning_model)


  | Name  | Type             | Params
-------------------------------------------
0 | model | Unet             | 31 M  
1 | CE    | CrossEntropyLoss | 0     


In [ ]:
plot_metrics(conf=config, metric='accuracy', scale='linear', save=True)

In [ ]:
trainer.test(
    model=lightning_model,
    ckpt_path = lightning_model.fm.model_checkpoint("best").as_posix()
)